In [1]:
import pandas as pd
import networkx as nx

In [2]:
from iac_sketch import data, etl, sketch, transform

In [48]:
# DEBUG
import importlib

importlib.reload(data)
importlib.reload(etl)
importlib.reload(transform)
importlib.reload(sketch)
# architect = sketch.Architect("../public/components")
# registry = architect.parse()

<module 'iac_sketch.sketch' from '/Users/zhafen/repos/iac-sketch/iac_sketch/sketch.py'>

In [49]:
import pandera.pandas as pa

In [50]:
extract_sys = etl.ExtractSystem()
registry = extract_sys.extract_entities("../public/components/*yaml")
registry

In [51]:
transform_sys = etl.TransformSystem()
registry = transform_sys.normalize_components(registry)
registry = transform_sys.extract_component_definitions(registry)

In [52]:
registry["component"]

,entity,comp_ind,multiplicity,comp_ind_fields,unparsed_fields,defined,fields,valid,errors
entity,,,,,,,,,
algorithm,algorithm,NaN,NaN,NaN,{},False,{},False,Component definition does not exist.
alternative,alternative,0.0,NaN,NaN,{},True,{},True,
alternative_to,alternative_to,NaN,NaN,NaN,{},False,{},False,Component definition does not exist.
append_to,append_to,NaN,NaN,NaN,{},False,{},False,Component definition does not exist.
associated_components,associated_components,NaN,NaN,NaN,{},False,{},False,Component definition does not exist.
...,...,...,...,...,...,...,...,...,...
url,url,NaN,NaN,NaN,{},False,{},False,Component definition does not exist.
used_in,used_in,NaN,NaN,NaN,{},False,{},False,Component definition does not exist.
website,website,NaN,NaN,NaN,{},False,{},False,Component definition does not exist.


In [53]:
component_key = "component"

In [54]:
component_def = registry["component"].loc[component_key]
component_def

entity                                                     component
comp_ind                                                         0.0
multiplicity                                                    0..1
comp_ind_fields                                                  2.0
unparsed_fields    {'comp_ind_fields [Int64]': {'description': 'C...
defined                                                         True
fields             {'comp_ind_fields': <Schema Field(name=comp_in...
valid                                                           True
errors                                                              
Name: component, dtype: object

In [55]:
schema = pa.DataFrameSchema(component_def["fields"])
schema

<Schema DataFrameSchema(columns={'comp_ind_fields': <Schema Field(name=comp_ind_fields, type=DataType(Int64))>, 'fields': <Schema Field(name=fields, type=None)>, 'unparsed_fields': <Schema Field(name=unparsed_fields, type=DataType(<class 'dict'>))>, 'defined': <Schema Field(name=defined, type=DataType(bool))>, 'valid': <Schema Field(name=valid, type=DataType(bool))>, 'errors': <Schema Field(name=errors, type=DataType(str))>, 'multiplicity': <Schema Field(name=multiplicity, type=DataType(str))>}, checks=[], parsers=[], index=None, dtype=None, coerce=False, strict=False, name=None, ordered=False, unique=None, report_duplicates=all, unique_column_names=False, add_missing_columns=False, title=None, description=None, metadata=None, drop_invalid_rows=False)>

In [59]:
try:
    valid_df = schema.validate(registry["component"])
except pa.errors.SchemaError as exc:
    print(exc)

In [60]:
valid_df

,entity,comp_ind,multiplicity,comp_ind_fields,unparsed_fields,defined,fields,valid,errors
entity,,,,,,,,,
algorithm,algorithm,NaN,0..*,<NA>,{},False,{},False,Component definition does not exist.
alternative,alternative,0.0,0..*,<NA>,{},True,{},True,
alternative_to,alternative_to,NaN,0..*,<NA>,{},False,{},False,Component definition does not exist.
append_to,append_to,NaN,0..*,<NA>,{},False,{},False,Component definition does not exist.
associated_components,associated_components,NaN,0..*,<NA>,{},False,{},False,Component definition does not exist.
...,...,...,...,...,...,...,...,...,...
url,url,NaN,0..*,<NA>,{},False,{},False,Component definition does not exist.
used_in,used_in,NaN,0..*,<NA>,{},False,{},False,Component definition does not exist.
website,website,NaN,0..*,<NA>,{},False,{},False,Component definition does not exist.


In [ ]:
# Create a directed graph from the DataFrame
graph = nx.from_pandas_edgelist(
    registry["link"],
    source="source",
    target="target",
    edge_key="link_type",
    create_using=nx.DiGraph,
)

# Visualize the graph
nx.draw(graph, with_labels=True)

In [ ]:
graph.add_nodes_from(registry["metadata"]["entity"].values)

In [ ]:
connected_components = [_ for _ in nx.connected_components(graph.to_undirected())]
metadata = registry["metadata"].set_index("entity")
metadata["connected_component_category"] = -1
for i, comps in enumerate(connected_components):
    metadata.loc[list(comps), "connected_component_category"] = i

In [ ]:
pair_connectivity = nx.all_pairs_node_connectivity(graph)

In [ ]:
sum(pair_connectivity["accept_request"].values()) > 0

In [ ]:
# Find invalid requirements
reqs = registry.view(["requirement", "status", "task", "input"])
is_valid = reqs["status"].isin(["closed", "removed"])
invalid_reqs = reqs.loc[~is_valid]
invalid_reqs

In [ ]:
# Find invalid testcases
registry.view(["testcase", "status"])

In [ ]:
import base64
import io, requests
from IPython.display import Image, display
from PIL import Image as im
import matplotlib.pyplot as plt

In [ ]:


def mm(graph):
    graphbytes = graph.encode("utf8")
    base64_bytes = base64.urlsafe_b64encode(graphbytes)
    base64_string = base64_bytes.decode("ascii")
    img = im.open(io.BytesIO(requests.get('https://mermaid.ink/img/' + base64_string).content))
    plt.imshow(img)
    plt.axis('off') # allow to hide axis

In [ ]:
graph = """
graph LR;
    A--> B & C & D
    B--> A & E
    C--> A & E
    D--> A
    E--> B & C & D
"""

In [ ]:
graphbytes = graph.encode("utf8")
base64_bytes = base64.urlsafe_b64encode(graphbytes)
base64_string = base64_bytes.decode("ascii")
result = requests.get('https://mermaid.ink/img/' + base64_string).content
parsed_result = io.BytesIO(result)
img = im.open(parsed_result)
img

In [ ]:
mm("""
flowchart LR;
    A--> B & C & D
    B--> A & E
    C--> A & E
    D--> A & E
    E--> B & C & D
""")